# Variables definition

In [6]:
import os
from dotenv import load_dotenv

if not load_dotenv("./../../../config/credentials_my.env", override=True):
    print("Environment variables not loaded :-(")
else:
    print("Environment variables have been loaded ;-)")

Environment variables have been loaded ;-)


In [1]:
import os
from dotenv import load_dotenv

if not load_dotenv("./../../../config/credentials_my.env", override=True):
    print("Environment variables not loaded :-(")
else:
    print("Environment variables have been loaded ;-)")
    
openai_endpoint       = os.environ["openai_endpoint"]
openai_api_key        = os.environ["openai_api_key"]
openai_api_version    = os.environ["openai_api_version"]
azure_deployment_name = os.environ["azure_deployment_name"]

apim_endpoint         = os.environ["apim_endpoint"]
apim_subscription_key = os.environ["apim_subscription_key_premium"] # apim_subscription_key_service, apim_subscription_key_standard and apim_subscription_key_premium
apim_api_version      = os.environ["apim_api_version"] # never used

# system and user message files
file_path = ".\\isp_docs\\"
system_message_path = f"{file_path}system_message.txt"
user_message_path = f"{file_path}user_message.txt"
answer_verification_path = f"{file_path}answer_verification.txt"

messages_sample = [
    {
      "role": "user",
      "content": "what is the nuclear fusion? explain in less than 20 words"
    }
]

KeyError: 'openai_endpoint'

# Prepare OpenAI call payload

In [2]:
def read_file_content(file_path: str) -> str:
    # Open and read the file
    try:
        with open(file_path, 'r') as file:
            # Read the contents of the file
            content = file.read()
            # Print the content
            return content
    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except IOError:
        print(f"An error occurred while reading the file at {file_path}.")


system_message = read_file_content(system_message_path)
user_message   = read_file_content(user_message_path)

messages = [
    {
      "role": "system",
      "content": system_message
    },
    {
      "role": "user",
      "content": user_message
    }
]

print(messages)

[{'role': 'system', 'content': 'id_fonte: 815\n contentuto_fonte:Cosa cambia dal 27 maggio 2023 per le promozioni rivolte agli Under 35?\nDal 27 maggio al 30 novembre 2023 i giovani usufruiscono di una promozione che azzera il canone mensile base del Xme Conto monointestato e il canone mensile della XME Card Plus se sottoscritta entro il 30 novembre 2023 fino alla fine del mese del compimento del trentacinquesimo anno di etÃ\xa0.\n\nid_fonte: 1203\n contentuto_fonte:A Chi si Rivolge\nLa carta puÃ² essere sottoscritta esclusivamente da maggiorenni che possiedono un Codice Fiscale valido e che abbiano sottoscritto il Servizio a distanza e firme elettroniche - My Key. In particolare, Superflash Ã¨ una carta prepagata nominativa ricaricabile destinata ai giovani, studenti o lavoratori, e a tutti coloro che desiderano disporre e ricevere bonifici (non Ã¨ possibile ricevere bonifici di importo inferiore a 1,00 euro), pagare presso gli esercizi commerciali e acquistare su Internet senza ancor

# Answer Check

In [4]:
def check_answer(answer: str) -> str:
    messages_verification = [
        {
            "role": "user",
            "content": read_file_content(answer_verification_path).replace("<string_to_check>", answer)
        }
    ]

    url = f"{openai_endpoint}openai/deployments/{azure_deployment_name}/chat/completions?api-version={openai_api_version}"
    response_http = requests.post(url, headers = {'api-key':openai_api_key}, json = {"messages": messages_verification})

    if (int(response_http.status_code / 100))==2:
        answer = response_http.json()['choices'][0]['message']['content']
        return(answer)
    else:
        print(response_http.text)

check_answer("è una carta prepagata nominativa") # "non lo so", "è una carta prepagata nominativa"

'WRONG'

# Invoking Open AI via HTTP or Azure SDK

In [5]:
def invoke_openai(method:str="http", iterations:int=10, temperature:float=0, top_p:float=0.95, max_tokens=500):

    filename = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_logs.txt") # Format the date and time as required
    correct_count = 0
    logs = ""
           
    if method == "http":
        messages_json = {"messages": messages, "temperature": temperature, "top_p": top_p, "max_tokens": max_tokens}
        url = f"{openai_endpoint}openai/deployments/{azure_deployment_name}/chat/completions?api-version={openai_api_version}"
    elif method == "sdk":
        client = AzureOpenAI(
            azure_endpoint = openai_endpoint,
            api_key        = openai_api_key,
            api_version    = openai_api_version
        )
        messages_json = messages
        parameters = {  
            "temperature": temperature,  # Adjust the temperature value as needed  
            "top_p": top_p,  # Adjust the top_p value as needed
            "max_tokens": max_tokens
        }

    for i in range(iterations):
        if method == "http":
            response_http = requests.post(url, headers = {'api-key':openai_api_key}, json = {"messages": messages})
            if (int(response_http.status_code / 100))==2:
                answer = response_http.json()['choices'][0]['message']['content']
            else:
                print(response_http.text)
                return
                
        elif method == "sdk":
            response_sdk = client.chat.completions.create(
                model    = azure_deployment_name,
                messages = messages_json,
                **parameters
            )
            answer = response_sdk.choices[0].message.content

        answer_result = check_answer(answer)
        correct_count += 1 if answer_result == "CORRECT" else 0

        output = f">>>> {i+1} ({method}) - {answer_result}: {answer}\n"
        print(output)
        logs = f"{logs}{output}\n"
    
    statistics = f"Summary: {correct_count} correct results out of {iterations} ({correct_count/iterations*100}%)"
    
    # Write the string to the file
    with open(f"{file_path}{filename}", "w") as file:
        file.write(f"{statistics}\n=======\n{logs}")
    
    print(statistics)

In [6]:
%%time

invoke_openai (method="sdk", iterations=10)

>>>> 1 (sdk) - CORRECT: Come Co-pilota dei Gestori di Filiale Digitale mi occupo di altri argomenti, per approfondirli consulta la Guida ai contenuti. Se vuoi chiedimi qualcos'altro e proverò a risponderti.

>>>> 2 (sdk) - CORRECT: Come Co-pilota dei Gestori di Filiale Digitale mi occupo di altri argomenti, per approfondirli consulta la Guida ai contenuti. Se vuoi chiedimi qualcos'altro e proverò a risponderti.

>>>> 3 (sdk) - CORRECT: Come Co-pilota dei Gestori di Filiale Digitale mi occupo di altri argomenti, per approfondirli consulta la Guida ai contenuti. Se vuoi chiedimi qualcos'altro e proverò a risponderti.

>>>> 4 (sdk) - CORRECT: Come Co-pilota dei Gestori di Filiale Digitale mi occupo di altri argomenti, per approfondirli consulta la Guida ai contenuti. Se vuoi chiedimi qualcos'altro e proverò a risponderti.

>>>> 5 (sdk) - WRONG: La carta giovani nazionale non è specificamente menzionata nelle fonti fornite. Tuttavia, posso fornirti informazioni su alcune carte destinate ai

In [7]:
%%time

invoke_openai (method="http", iterations=10)

>>>> 1 (http) - WRONG: La Carta Superflash è una carta prepagata nominativa ricaricabile destinata ai giovani, studenti o lavoratori. Consente di disporre e ricevere bonifici (non è possibile ricevere bonifici di importo inferiore a 1,00 euro), pagare presso gli esercizi commerciali e acquistare su Internet senza possedere un conto corrente. Non può essere rilasciata a minori o incapaci. [1203]

**Funzioni disponibili**
- Gestione notifiche per ricevere un SMS, notifica su app o e-mail per ogni operazione oltre le soglie scelte. 
- Possibilità di bloccare o sbloccare la carta tramite app, banca online e con la filiale online.
- Possibilità di recuperare il PIN tramite app, banca online e con la filiale online.
- Elevata sicurezza negli acquisti online con il servizio Pagamenti sicuri internet.
- Servizio GeoControl per limitare l'utilizzo della carta a seconda del paese in cui ti trovi. 
[1396]

>>>> 2 (http) - WRONG: La carta Superflash è una carta prepagata nominativa ricaricabile de